In [ ]:
import holoviews as hv
import numpy as np
from holoviews import opts
hv.extension('bokeh')
from bokeh.plotting import show

In [ ]:
def anti_clockwise(a, b, c):
    prod = (b[0] - a[0]) * (c[1] - a[1]) - (b[1] - a[1]) * (c[0] - a[0])
    return True if prod > 0 else False

def in_triangle(p, q, r, s):
    d1 = anti_clockwise(q, r, p)
    d2 = anti_clockwise(r, s, p)
    d3 = anti_clockwise(s, q, p)
    return True if (d1 and d2 and d3) else False

def ears_in_polygon(polygon):
    size = len(polygon)
    ears = []
    if size < 3:
        return ears
    if size == 3:
        return polygon
    for vertex in range(size):
        if is_ear(polygon, vertex):
            ears.append(polygon[vertex % size])
    return ears

def is_ear(polygon, vertex):
    size = len(polygon)
    a = polygon[(vertex-1) % size].xy
    b = polygon[vertex % size].xy
    c = polygon[(vertex+1) % size].xy
    if not anti_clockwise(a, b, c):
        return False
    for other_vertex in polygon:
        if in_triangle(other_vertex.xy, a, b, c):
            return False
    return True

def ear_clipping(polygon):
    print('Ear-Clipping algorithm has started')
    print()
    
    original_polygon = polygon.copy()
    edges = []
    graph = []
    ears = ears_in_polygon(polygon)
    
    for i in range(0,len(polygon)):
        vertex_num = polygon[i].num
        vertex_next_num = polygon[(i+1) % len(polygon)].num
        edges.append((vertex_next_num, vertex_num))
    
    for edge in edges:
        graph.append({'x': [polygon[edge[0]].xy[0], polygon[edge[1]].xy[0]], 'y': [polygon[edge[0]].xy[1], polygon[edge[1]].xy[1]], 'value':'black'})
    
    print('Polygon:')
    path = hv.Path(graph, vdims='value').opts(color='value', line_width=1.5)
    plot = hv.render(path)
    show(plot)
    
    i=0
    while (len(polygon) > 3 and len(ears) > i):
        print('Ears at the moment:')
        for j in range(i, len(ears)):
            print(ears[j].num, end = " ")
        print()
        
        vertex_index = polygon.index(ears[i])
        vertex_next_index = (vertex_index+1) % len(polygon)
        vertex_before_index = (vertex_index-1) % len(polygon)
        vertex_num = polygon[vertex_index].num
        vertex_next_num = polygon[vertex_next_index].num
        vertex_before_num = polygon[vertex_before_index].num
        
        if (not (polygon[vertex_before_index].xy[0] == polygon[vertex_index].xy[0] and polygon[vertex_next_index].xy[0] == polygon[vertex_index].xy[0])):
            if (not (polygon[vertex_before_index].xy[1] == polygon[vertex_index].xy[1] and polygon[vertex_next_index].xy[1] == polygon[vertex_index].xy[1])):
                edges.append((vertex_before_num, vertex_next_num))
        graph.append({'x': [original_polygon[vertex_before_num].xy[0], original_polygon[vertex_next_num].xy[0]], 'y': [original_polygon[vertex_before_num].xy[1], original_polygon[vertex_next_num].xy[1]], 'value':'black'})
        
        print("Removed vertex {} (painted orange)".format(vertex_num))
        
        print("Analysing vertices {} and {} (painted black)".format(vertex_before_num, vertex_next_num))
        for edge in graph:
            for k in range(len(edge['x'])):
                if edge['x'][k] == original_polygon[vertex_num].xy[0] and original_polygon[vertex_num].xy[1] == edge['y'][k]:
                    edge['value'] = 'orange'
        points = hv.Points([polygon[vertex_next_index].xy, polygon[vertex_before_index].xy]).opts(color='black', size=10)
        point = hv.Points([polygon[vertex_index].xy]).opts(color='orange', size=10)
        path = path = hv.Path(graph, vdims='value').opts(color='value', line_width=1.5)*points*point
        plot = hv.render(path)
        show(plot)
        
        polygon.pop(vertex_index)
        vertex_before_index = (vertex_index-1) % len(polygon)
        vertex_before = polygon[vertex_before_index]
        vertex_next_index = vertex_index % len(polygon)
        vertex_next = polygon[vertex_next_index]
        if(is_ear(polygon, vertex_before_index) and vertex_before not in ears):
            ears.append(vertex_before)
        if(is_ear(polygon, vertex_next_index) and vertex_next not in ears):
            ears.append(vertex_next)
        i += 1
        
    print('Ear-Clipping algorithm has reached its end')
    return (edges, graph, original_polygon)

In [ ]:
def three_colouring(edges, graph, polygon):
    print('3-Colouring algorithm has started')
    size = len(polygon)
    parity = [0] * size
    for vertex in range(size):
            for edge in edges:
                if edge[0] == vertex or edge[1] == vertex:
                    parity[vertex] += 1
    color = [0] * size
    color = [0] * size
    color[0] = 1
    color[1] = 2
    for i in range(1, size-1):
        if parity[i] % 2 != 0:
            color[i+1] = color[i-1]
        else:
            color[i+1] = 6 - color[i-1] - color[i]
    points_blue, points_red, points_green = [],[],[]
    for i in range(len(color)):
        if color[i] == 1:
            points_blue.append(polygon[i].xy)
        elif color[i] == 2:
            points_red.append(polygon[i].xy)
        elif color[i] == 3:
            points_green.append(polygon[i].xy)
    points = hv.Points(points_blue).opts(color='blue', size=10)*hv.Points(points_red).opts(color='red', size=10)*hv.Points(points_green).opts(color='green', size=10)
    path = hv.Path(graph).opts(color='black', line_width=1.5)*points
    plot = hv.render(path)
    show(plot)
    print('3-Colouring algorithm has reached its end')

Source of the three_colouring algorithm: https://www.researchgate.net/publication/256309448_Three-coloring_the_vertices_of_a_triangulated_simple_polygon

In [ ]:
class Point:

    def __init__(self, num, xy):
        self.num = num
        self.xy = xy

# User-friendly main method
'''
def main():
    polygon = []
    num_vertices = int(input("Insert number of vertices of the polygon: "))
    print("Insert coordinates of vertices in format x,y and in anticlockwise order!!!")
    for i in range(num_vertices):
        point = input("Vertex {}: ".format(i))
        xy = tuple(map(int, point.split(',')))
        polygon.append(Point(i,xy))
    edges, graph, polygon = ear_clipping(polygon)
    three_colouring(edges, graph, polygon)
'''
    
# Mocked polygon main method
#'''
def main():
    polygon = []
    mock = [(0,1),(1,1),(2,2),(2,3)] # Here's your polygon vertices in anticlockwise order
    num_vertices = len(mock)
    for i in range(num_vertices):
        xy = mock[i]
        polygon.append(Point(i,xy))
    edges, graph, polygon = ear_clipping(polygon)
    three_colouring(edges, graph, polygon)
#'''

In [ ]:
main()
# TRY THESE
# Polygon 1: (0,5),(8,0),(18,7),(26,3),(28,17),(19,13),(17,20),(11,6),(4,9),(3,19)
# Polygon 2: (17,15),(20,10),(25,13),(23,20),(13,14),(10,20),(11,11)
# Polygon 3: (0, 0),(12, 0),(11.0, 10),(10.5, 2),(10.0, 2),(9.5, 10),(9.0, 2),
#        (8.5, 2),(8.0, 10),(7.5, 2),(7.0, 2),(6.5, 10),
#        (6.0, 2),(5.5, 2),(5.0, 10),(4.5, 2),(4.0, 2),(3.5, 10),(3.0, 2),
#        (2.5, 2),(2.0, 10),(1.5, 2),(1.0, 2),(0.5, 10)